In [1]:
import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv('train.csv')
df100 = df.head(1000)
df1 = df.copy()
df2 = df.copy()
df3 = df.copy()
df4 = df3.head(10000)
df1.shape
df3True = df3.loc[df3['target'] == 1]
df3False = df3.loc[df3['target'] != 1]


In [31]:
df4.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [32]:

df1['word_count'] = df1['question_text'].str.split().str.len()                # Number of words per questions
df1['Uppercase'] = df1['question_text'].str.findall(r'[A-Z]').str.len()       # Number of upper case letters
df1['Lowercase'] = df1['question_text'].str.findall(r'[a-z]').str.len()       # Number of lowercase letters
df1['No_of_QuestionMarks'] = df1['question_text'].str.findall(r'\?').str.len()# Number of question Marks
df1.head()

,qid,question_text,target,word_count,Uppercase,Lowercase,No_of_QuestionMarks
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0,13,2,53,1
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0,16,1,63,1
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0,10,2,54,2
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0,9,4,44,1
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0,15,3,59,1


In [33]:
from nltk.corpus import stopwords
stop = stopwords.words('english')


In [34]:
## extract stop words and count them
df1['stop_words'] = df1['question_text'].apply(lambda x: [item for item in x.split() if item in stop])
df1['stop_words_len'] = df1['stop_words'].str.len()
df1.head()

,qid,question_text,target,word_count,Uppercase,Lowercase,No_of_QuestionMarks,stop_words,stop_words_len
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0,13,2,53,1,"[did, their, as, a, in, the]",6
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0,16,1,63,1,"[you, have, an, how, you, to, and, not]",8
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0,10,2,54,2,[does],1
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0,9,4,44,1,"[did, the]",2
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0,15,3,59,1,"[to, a, by, just, the]",5


## Encode question types as a one-hot encoder

In [35]:
firstWordTrue = df3True['question_text'].str.split().str[0]
firstWordFalse = df3False['question_text'].str.split().str[0]


In [36]:
TrueCount = firstWordTrue.value_counts().index.tolist()[:20]
FalseCount = firstWordFalse.value_counts().index.tolist()[:20]

In [37]:
def countStartWord_anywhereT(x):
    zeros = np.zeros(len(TrueCount))
    for item in x.split():
        if item in TrueCount:
            zeros[TrueCount.index(item)] = 1
            return zeros
    return zeros


def countStartWord_onlyBeginingT(x):
    zeros = np.zeros(len(TrueCount))
    if x.split()[0] in TrueCount:
        zeros[TrueCount.index(x.split()[0])] = 1
        return zeros
    return zeros

def countStartWordOccurenceT(x):
    count = 0
    for item in x.split():
        if item in TrueCount:
            count = count + 1
    return count

def countStartWord_anywhereF(x):
    zeros = np.zeros(len(FalseCount))
    for item in x.split():
        if item in FalseCount:
            zeros[FalseCount.index(item)] = 1
            return zeros
    return zeros


def countStartWord_onlyBeginingF(x):
    zeros = np.zeros(len(FalseCount))
    if x.split()[0] in FalseCount:
        zeros[FalseCount.index(x.split()[0])] = 1
        return zeros
    return zeros

def countStartWordOccurenceF(x):
    count = 0
    for item in x.split():
        if item in FalseCount:
            count = count + 1
    return count

In [38]:
df1['countStartWord_anywhere_inText'] = -1
df1['countStartWord_anywhere_inText'].loc[df3['target'] == 1] = df3True['question_text'].apply(countStartWord_anywhereT)
df1['countStartWord_anywhere_inText'].loc[df3['target'] == 0] = df3False['question_text'].apply(countStartWord_anywhereF)

In [39]:
df1['countStartWord_Occurence'] = -1
df1['countStartWord_Occurence'].loc[df3['target'] == 1] = df3True['question_text'].apply(countStartWordOccurenceT)
df1['countStartWord_Occurence'].loc[df3['target'] == 0] = df3False['question_text'].apply(countStartWordOccurenceF)

In [40]:
df1['countStartWord_onlyAtStart_inText'] = -1
df1['countStartWord_onlyAtStart_inText'].loc[df3['target'] == 1] = df3True['question_text'].apply(countStartWord_onlyBeginingT)
df1['countStartWord_onlyAtStart_inText'].loc[df3['target'] == 0] = df3False['question_text'].apply(countStartWord_onlyBeginingF)

In [41]:
df1.head()

,qid,question_text,target,word_count,Uppercase,Lowercase,No_of_QuestionMarks,stop_words,stop_words_len,countStartWord_anywhere_inText,countStartWord_Occurence,countStartWord_onlyAtStart_inText
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0,13,2,53,1,"[did, their, as, a, in, the]",6,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0,16,1,63,1,"[you, have, an, how, you, to, and, not]",8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0,10,2,54,2,[does],1,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0,9,4,44,1,"[did, the]",2,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0,15,3,59,1,"[to, a, by, just, the]",5,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",2,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."


## Encode question N-grams

In [42]:
secondWordTrue2 = df3True['question_text'].str.split().str[1]
secondWordFalse2 = df3False['question_text'].str.split().str[1]

In [43]:
TrueCount2 = secondWordTrue2 .value_counts().index.tolist()[:20]
FalseCount2 = secondWordFalse2.value_counts().index.tolist()[:20]

In [44]:
import itertools
TrueCountNGram = []
for x in itertools.product(TrueCount,TrueCount2):
    k = ' '.join(x)
    TrueCountNGram.append(k)

import itertools
FalseCountNGram = []
for x in itertools.product(FalseCount,FalseCount2):
    k = ' '.join(x)
    TrueCountNGram.append(k)

In [37]:
TrueCountNGram

['Why do',
 'Why are',
 'Why is',
 'Why the',
 'Why it',
 'Why does',
 'Why you',
 'Why can',
 'Why did',
 'Why a',
 "Why don't",
 'Why I',
 'Why will',
 'Why there',
 'Why would',
 'Why many',
 'Why Trump',
 'Why we',
 'Why Indian',
 "Why can't",
 'Is do',
 'Is are',
 'Is is',
 'Is the',
 'Is it',
 'Is does',
 'Is you',
 'Is can',
 'Is did',
 'Is a',
 "Is don't",
 'Is I',
 'Is will',
 'Is there',
 'Is would',
 'Is many',
 'Is Trump',
 'Is we',
 'Is Indian',
 "Is can't",
 'How do',
 'How are',
 'How is',
 'How the',
 'How it',
 'How does',
 'How you',
 'How can',
 'How did',
 'How a',
 "How don't",
 'How I',
 'How will',
 'How there',
 'How would',
 'How many',
 'How Trump',
 'How we',
 'How Indian',
 "How can't",
 'What do',
 'What are',
 'What is',
 'What the',
 'What it',
 'What does',
 'What you',
 'What can',
 'What did',
 'What a',
 "What don't",
 'What I',
 'What will',
 'What there',
 'What would',
 'What many',
 'What Trump',
 'What we',
 'What Indian',
 "What can't",
 'Do do'

In [45]:

def get_ngrams(text):
    n_grams = ngrams(word_tokenize(text), 2)
    return [ ' '.join(grams) for grams in n_grams]

def countStartWordNGram_anywhereT(x):
    zeros = np.zeros(len(TrueCountNGram))
    for item in get_ngrams(x):
        if item in TrueCountNGram:
            zeros[TrueCountNGram.index(item)] = 1
            return zeros
    return zeros


def countStartWordNGram_onlyBeginingT(x):
    zeros = np.zeros(len(TrueCountNGram))
    if len(get_ngrams(x))>0:
        if get_ngrams(x)[0] in TrueCountNGram:
            zeros[TrueCountNGram.index(get_ngrams(x)[0])] = 1
            return zeros
    return zeros

def countStartWordOccurenceNGramT(x):
    count = 0
    for item in get_ngrams(x):
        if item in TrueCountNGram:
            count = count + 1
    return count

from nltk.tokenize import word_tokenize
from nltk.util import ngrams




def countStartWordNGram_anywhereF(x):
    zeros = np.zeros(len(FalseCountNGram))
    for item in get_ngrams(x):
        if item in FalseCountNGram:
            zeros[FalseCountNGram.index(item)] = 1
            return zeros
    return zeros


def countStartWordNGram_onlyBeginingF(x):
    zeros = np.zeros(len(FalseCountNGram))
    if len(get_ngrams(x))>0:
        if get_ngrams(x)[0] in FalseCountNGram:
            zeros[FalseCountNGram.index(get_ngrams(x)[0])] = 1
            return zeros
    return zeros

def countStartWordOccurenceNGramF(x):
    count = 0
    for item in get_ngrams(x):
        if item in FalseCountNGram:
            count = count + 1
    return count

from nltk.tokenize import word_tokenize
from nltk.util import ngrams


# def get_ngrams(text):
#     n_grams = ngrams(word_tokenize(text), 2)
#     return [ ' '.join(grams) for grams in n_grams]

In [46]:
df1['count2gram_anywhere_inText'] = -1
df1['count2gram_anywhere_inText'].loc[df3['target'] == 1] = df3True['question_text'].apply(countStartWordNGram_anywhereT)
df1['count2gram_anywhere_inText'].loc[df3['target'] == 0] = df3False['question_text'].apply(countStartWordNGram_anywhereF)

In [47]:
df1['count2gram_Occurence'] = -1
df1['count2gram_Occurence'].loc[df3['target'] == 1] = df3True['question_text'].apply(countStartWordOccurenceNGramT)
df1['count2gram_Occurence'].loc[df3['target'] == 0] = df3False['question_text'].apply(countStartWordOccurenceNGramF)

In [49]:
df1['count2gram_onlyAtStart_inText'] = -1
df1['count2gram_onlyAtStart_inText'].loc[df3['target'] == 1] = df3True['question_text'].apply(countStartWordNGram_onlyBeginingT)
df1['count2gram_onlyAtStart_inText'].loc[df3['target'] == 0] = df3False['question_text'].apply(countStartWordNGram_onlyBeginingF)

KeyboardInterrupt: 

In [ ]:
df1.head()
